# Loading a mesh from STL file(binary)
 - STL 은 제품의 표면 기하 정보를 삼각형으로 표현함
 - STL 파일에는 스케일이나 단위 정보, 컬러 정보 등이 포함되어 있지 않음
 - 보통 빠르게 3D 프린팅 등을 통해 빠르게 프로토타이핑 하는데 사용됨

In [1]:
import pyvista as pv
pv.global_theme.jupyter_backend = 'client'
pv.start_xvfb()

# STL(Stereolithography CAD) 파일의 구조
 - https://en.wikipedia.org/wiki/STL_(file_format)
 - An STL file describes a raw, unstructured triangulated surface by the unit normal and vertices (ordered by the right-hand rule[2]) of the triangles

In [2]:
# ASCII 또는 Binary 형식으로 저장된 STL 파일을 읽어서 표시합니다.
stl_filepath = '../../data/10_30.stl'

In [3]:
with open('../../data/10_30.stl', 'rb') as f:
    header = f.read(80)
    header_str = header.decode('utf-8')
    print(header_str)
    n_triangles = int.from_bytes(f.read(4), 'little')
    print(n_triangles)

Visualization Toolkit generated SLA File                                        
35750


 - 파일을 직접 파싱하여 triangle 개수를 직접 확인(예: 35,750)

In [4]:
mesh = pv.read(stl_filepath)

In [5]:
type(mesh)

pyvista.core.pointset.PolyData

# PolyData 클래스
 - 3D Surface Mesh 에 대한 정보를 표현하는 core class
 - Mesh 에 대한 복제, 시각화, 처리(decimate, smoothing, rotate, sampling 등) 지원

In [6]:
mesh

PolyData,Information
N Cells,35750
N Points,17859
N Strips,0
X Bounds,"-3.881e+01, 6.821e+01"
Y Bounds,"-1.608e+02, 1.780e+01"
Z Bounds,"2.534e-01, 6.527e+01"
N Arrays,0


## PolyData 의 핵심 속성
 - N Cells: cell의 개수
    - cell은 PolyData의 기본 구성 단위 요소로 삼각형, 선분, 점을 모두 포함함
    - face는 표면에 존재하는 다각형 요소만을 의미하므로 삼각형, 사각형 등을 의미함(선분과 점이 빠짐)
    - 즉 모든 face는 cell 이지만, cell이 face인 것은 아님
    - STL 파일에서는 일반적으로 표면을 삼각형으로 표현하기 때문에 3개의 꼭짓점 다른 말로 정점(vertex)를 가짐
 - N Points: 정점의 개수
    - 삼각형 셀들은 정점들을 공유할 수 있음
    - 예를 들어 두 삼각형이 같은 정점을 사용하는 것이 가능
    - 때문에 정점의 수는 삼각형 수보다 작을 수 있음(일반적)
 - N Strips: 삼각형을 배치할 때 공간 효율을 위해 사용하는 스트립의 개수
    - STL 에서는 사용되지 않음
 - X/Y/Z Bounds: 각 축의 경계 범위(bounding box)
    - 즉 x, y, z 축의 최소, 최대 값들
    - 이 값은 STL에 저장되어 있지 않고, 로드 시 계산 됨
 - N Arrays: 데이터 배열
    - 정점 혹은 면에 할당 된 변수(예: color, normal 등)들
    - STL은 보통 단순 Geometry만 포함하므로 데이터 배열이 없음

In [58]:
len(dir(mesh))

524

In [59]:
len([i for i in dir(mesh) if i[0].islower()])

216


# PolyData의 상세 속성
 - 정말 많은 속성이 정의되어 있음
 - 소문자로 시작하는 속성이 자주 사용될 수 있는 것들인데, 이것들만 해도 216개 임
 - 아래 표는 그 중 추려 본 것

| 속성 이름        | 설명                                                       |
| ------------ | -------------------------------------------------------- |
| `points`     | 정점(vertices) 좌표를 포함하는 NumPy 배열 (`(N, 3)` 형태)             |
| `cells`      | 셀(삼각형, 사각형 등)의 정점 인덱스 정보                                 |
| `faces`      | PolyData의 면(faces) 정의 배열 (특히 삼각형/사각형)                    |
| `n_points`   | 정점 개수 (`len(mesh.points)`)                               |
| `n_cells`    | 셀 개수 (`len(mesh.cells)`)                                 |
| `bounds`     | 메시의 경계값 (x\_min, x\_max, y\_min, y\_max, z\_min, z\_max) |
| `point_data` | 각 정점에 연결된 데이터 (예: 색상, 벡터 등)                              |
| `cell_data`  | 각 셀에 연결된 데이터                                             |
| `field_data` | 전체 메시에 연결된 데이터 (메타데이터 등)                                 |
| `lines`      | 선 정보가 있을 경우 해당 데이터                                       |
| `volume`     | 메시의 부피 (닫힌 경우에만 유의미)                                     |
| `area`       | 메시의 표면적                                                  |

## cell_data 와 field_data 비교
| 항목            | `cell_data`                        | `field_data`               |
| ------------- | ---------------------------------- | -------------------------- |
| **정의**        | 각 \*\*셀(cell)\*\*에 붙은 데이터          | 메시 전체에 붙은 **전역적인 메타데이터**   |
| **대상 단위**     | 메시의 각 셀 (예: 삼각형 하나)                | 메시 전체 또는 외부 연산용 정보         |
| **자료 구조**     | `DataSetAttributes` (key-value 형태) | `FieldData` (key-value 형태) |
| **사용 예시**     | 셀마다 재질 번호, 품질 지표 등 저장              | 파일 메타정보, 전처리 상태 플래그 등      |
| **크기 제약**     | 데이터의 길이는 `mesh.n_cells`와 같아야 함     | 제약 없음 (자유로운 배열 구조 가능)      |
| **시각화 반영**    | 시각화에서 셀 색상 등으로 표현 가능               | 시각화에 직접 반영되지 않음            |
| **파일 I/O 영향** | 대부분 포맷에 저장됨                        | 일부 포맷(STL 등)에서는 무시될 수 있음   |

## 메시의 부피와 닫힘(watertight)의 의미
 - 메시에 구멍(hole)이나 틈(gap)이 없다는 의미
 - 즉 모든 가장자리(edge)가 두 면(faces) 이상으로 공유된다는 것을 의미
 - volume 은 메시가 닫힘 상태 일 때만 계산 가능
 - 시각화 시에는 잘 보이지 않는 틈이 있는 경우 주의 해야 함


In [52]:
mesh.is_manifold

True

## 메시가 Manifold 하다는 것의 의미(is_manifold 가 True)
 - 모든 가장자리(edge)가 두 개 면에만 공유
 - 정점 주변이 연속적임(링 또는 팬을 구성)
 - 메시가 깨끗하게 잘 붙어 있는 표면, 즉 구멍이나 꼬인 곳 없이 연속적이고 매끄러운 상태를 의미
 - 3D Printing, 시뮬레이션에 있어서 중요

In [9]:
vtk_file_path = '../../data/10_30.vtk'
mesh.save(vtk_file_path)

# VTK 파일 형식(version:5.1)
 - https://docs.vtk.org/en/latest/design_documents/VTKFileFormats.html
 - ASCII 로 구성된 5줄짜리 헤더 + 데이터 영역
 - 헤더에서 파일 형식이 ASCII, Binary, XML 인지 판단
 - 데이터 영역은 크게 POINTS, CELLS 섹션으로 나뉨

In [17]:
import struct

def binary_to_float(binary_data, byteorder='little'):
    """바이너리 데이터를 float로 변환"""
    if len(binary_data) != 4:
        raise ValueError("float32는 4바이트가 필요합니다")
    
    # struct 모듈 사용
    if byteorder == 'little':
        return struct.unpack('<f', binary_data)[0]
    else:
        return struct.unpack('>f', binary_data)[0]


In [36]:
vtk_file_path = '../../data/10_30.vtk'
with open(vtk_file_path, 'rb') as f:
    lines = []
    
    # ASCII 헤더 파트(첫 5줄)
    for _ in range(5):
        line = f.readline().decode('utf-8').strip()
        lines.append(line)
    
    # header 정보 구조
    version = lines[0]
    description = lines[1]
    format_type = lines[2].upper()
    dataset_type = lines[4]
    
    # print header 정보
    print(f"Version: {version}")
    print(f"Description: {description}")
    print(f"Format Type: {format_type}")
    print(f"Dataset Type: {dataset_type}")

    # 바이너리 데이터 파트 읽기
    tokens = dataset_type.split()
    d_name = tokens[0]
    d_len = int(tokens[1])
    d_type = tokens[2]

    # POINTS 섹션 로드
    # for _ in range(d_len):
    #     x = binary_to_float(f.read(4), 'little')
    #     y = binary_to_float(f.read(4), 'little')
    #     z = binary_to_float(f.read(4), 'little')
    # POINTS 섹션 스킵
    f.seek(d_len * 4 * 3, 1)
    
    # CELLS 헤더 로드
    lines = []
    for _ in range(3):
        line = f.readline().decode('utf-8').strip()
        lines.append(line)
    
    split_line = lines[0]
    polygons = lines[1]
    offset = lines[2]

    polygon_tokens = polygons.split()
    n_polygons = int(polygon_tokens[1])
    print(f"Number of Polygons: {n_polygons}")
    
    # CELLS 데이터 로드

Version: # vtk DataFile Version 5.1
Description: vtk output
Format Type: BINARY
Dataset Type: POINTS 17859 float
Number of Polygons: 35751
